# Техническая часть

In [0]:
%matplotlib inline
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Постановка задачи и Подготовка данных

Буду реализововать Collaborative filtering.

In [0]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [0]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27753444 entries, 0 to 27753443
Data columns (total 4 columns):
userId       int64
movieId      int64
rating       float64
timestamp    int64
dtypes: float64(1), int64(3)
memory usage: 847.0 MB


In [0]:
movies[40:60]

,movieId,title,genres
40,41,Richard III (1995),Drama|War
41,42,Dead Presidents (1995),Action|Crime|Drama
42,43,Restoration (1995),Drama
43,44,Mortal Kombat (1995),Action|Adventure|Fantasy
44,45,To Die For (1995),Comedy|Drama|Thriller
45,46,How to Make an American Quilt (1995),Drama|Romance
46,47,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
47,48,Pocahontas (1995),Animation|Children|Drama|Musical|Romance
48,49,When Night Is Falling (1995),Drama|Romance
49,50,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [0]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58098 entries, 0 to 58097
Data columns (total 3 columns):
movieId    58098 non-null int64
title      58098 non-null object
genres     58098 non-null object
dtypes: int64(1), object(2)
memory usage: 1.3+ MB


Данные являются табличными и состоят из идентификатора пользователя, идентификатора фильма и рейтинга (есть также временная метка, но мы не будем использовать ее для этой задачи). Наша задача-предсказать рейтинг для пары пользователь / фильм, с идеей, что если бы у нас была модель, которая хороша в этой задаче, мы могли бы предсказать, как пользователь будет оценивать фильмы, которые они еще не видели, и рекомендовать фильмы с самым высоким прогнозируемым рейтингом.

Построим матрицу предпочтений, чтобы лучше понять, как вяглядят данные:

In [0]:
g = ratings.groupby('userId')['rating'].count()
top_users = g.sort_values(ascending=False)[:15]

g = ratings.groupby('movieId')['rating'].count()
top_movies = g.sort_values(ascending=False)[:15]

top_r = ratings.join(top_users, rsuffix='_r', how='inner', on='userId')
top_r = top_r.join(top_movies, rsuffix='_r', how='inner', on='movieId')

pd.crosstab(top_r.userId, top_r.movieId, top_r.rating, aggfunc=np.sum) 

movieId,1,50,110,260,296,318,356,480,527,589,593,1196,2571,2858,2959
userId,,,,,,,,,,,,,,,
68,2.5,3.0,2.5,5.0,2.0,3.0,3.5,3.5,4.0,3.5,3.5,5.0,4.5,5.0,2.5
182,4.0,4.5,3.5,3.5,5.0,4.5,5.0,3.5,4.0,2.0,4.5,3.0,5.0,5.0,5.0
249,4.0,4.0,5.0,5.0,4.0,4.5,4.5,4.0,4.5,4.0,4.0,5.0,5.0,4.5,5.0
274,4.0,4.0,4.5,3.0,5.0,4.5,4.5,3.5,4.0,4.5,4.0,4.5,4.0,5.0,5.0
288,4.5,NaN,5.0,5.0,5.0,5.0,5.0,2.0,5.0,4.0,5.0,4.5,3.0,NaN,3.5
307,4.0,4.5,3.5,3.5,4.5,4.5,4.0,3.5,4.5,2.5,4.5,3.0,3.5,4.0,4.0
380,5.0,4.0,4.0,5.0,5.0,3.0,5.0,5.0,NaN,5.0,5.0,5.0,4.5,NaN,4.0
387,NaN,4.5,3.5,4.5,5.0,3.5,4.0,3.0,NaN,3.5,4.0,4.5,4.0,4.5,4.5
414,4.0,5.0,5.0,5.0,5.0,5.0,5.0,4.0,4.0,5.0,4.0,5.0,5.0,5.0,5.0


In [0]:
user_enc = LabelEncoder()
ratings['user'] = user_enc.fit_transform(ratings['userId'].values)
n_users = ratings['user'].nunique()

item_enc = LabelEncoder()
ratings['movie'] = item_enc.fit_transform(ratings['movieId'].values)
n_movies = ratings['movie'].nunique()

ratings['rating'] = ratings['rating'].values.astype(np.float32)
min_rating = min(ratings['rating'])
max_rating = max(ratings['rating'])

n_users, n_movies, min_rating, max_rating

(610, 9724, 0.5, 5.0)

Создадим классические X и y, чтобы потом разделить на обучающую и тестовую выборку

In [0]:
ratings.head()

,userId,movieId,rating,timestamp,user,movie
0,1,1,4.0,964982703,0,0
1,1,3,4.0,964981247,0,2
2,1,6,4.0,964982224,0,5
3,1,47,5.0,964983815,0,43
4,1,50,5.0,964982931,0,46


In [0]:
X = ratings[['user', 'movie']].values
y = ratings['rating'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((90752, 2), (10084, 2), (90752,), (10084,))

Еще одна константа, которая нам понадобится для модели, - это количество факторов на пользователя / фильм. Это число может быть любым, однако для модели коллаборативной фильтрации оно должно быть одинакового размера как для пользователей, так и для фильмов

Так как мы будем строить нейронную сеть в двумя входами, разделим наши данные на два входа.

In [0]:
n_factors = 50

X_train_array = [X_train[:, 0], X_train[:, 1]]
X_test_array = [X_test[:, 0], X_test[:, 1]]

In [0]:
from keras.models import Model
from keras.layers import Input, Reshape, Dot
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.regularizers import l2

Using TensorFlow backend.


In [0]:
def RecommenderV1(n_users, n_movies, n_factors):
    user = Input(shape=(1,))
    u = Embedding(n_users, n_factors, embeddings_initializer='he_normal',
                  embeddings_regularizer=l2(1e-6))(user)
    u = Reshape((n_factors,))(u)
    
    movie = Input(shape=(1,))
    m = Embedding(n_movies, n_factors, embeddings_initializer='he_normal',
                  embeddings_regularizer=l2(1e-6))(movie)
    m = Reshape((n_factors,))(m)
    
    x = Dot(axes=1)([u, m])

    model = Model(inputs=[user, movie], outputs=x)
    opt = Adam(lr=0.001)
    model.compile(loss='mean_squared_error', optimizer=opt)

    return model

In [0]:
model = RecommenderV1(n_users, n_movies, n_factors)
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1, 50)        30500       input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 1, 50)        486200      input_2[0][0]                    
_____________________________________

In [0]:
history = model.fit(x=X_train_array, y=y_train, batch_size=32, epochs=10,
                    verbose=1, validation_data=(X_test_array, y_test))

Instructions for updating:
Use tf.cast instead.
Train on 90752 samples, validate on 10084 samples
Epoch 1/10
90752/90752 [==============================] - 35s 384us/step - loss: 7.5483 - val_loss: 2.3487
Epoch 2/10
90752/90752 [==============================] - 35s 384us/step - loss: 1.5464 - val_loss: 1.4633
Epoch 3/10
90752/90752 [==============================] - 33s 368us/step - loss: 0.9897 - val_loss: 1.3151
Epoch 4/10
90752/90752 [==============================] - 34s 375us/step - loss: 0.7948 - val_loss: 1.2677
Epoch 5/10
90752/90752 [==============================] - 34s 375us/step - loss: 0.6769 - val_loss: 1.2462
Epoch 6/10
90752/90752 [==============================] - 34s 371us/step - loss: 0.5870 - val_loss: 1.2566
Epoch 7/10
90752/90752 [==============================] - 34s 370us/step - loss: 0.5132 - val_loss: 1.2528
Epoch 8/10
90752/90752 [==============================] - 34s 374us/step - loss: 0.4439 - val_loss: 1.2569
Epoch 9/10
90752/90752 [======================

In [0]:
X_test[:10, 0], X_test[:10, 1]

(array([431, 287, 598,  41,  74,  50, 353, 415, 437,  72]),
 array([7316,  412, 3217, 2248, 1210,  149, 6416,  602, 4403, 5253]))

In [0]:
a = np.array([[100], [44]])
b = np.array(44)

In [0]:
model.predict([a[0], a[1]])

array([[3.0242043]], dtype=float32)

Вот такой рейтинг будет спрогназирован для 100 пользователя, я бы не рекомендовал этот фильм. 